In [ ]:
from transformers import BertForSequenceClassification, BertTokenizer
import torch

# initialize our model and tokenizer
tokenizer = BertTokenizer.from_pretrained('ProsusAI/finbert')
model = BertForSequenceClassification.from_pretrained('ProsusAI/finbert')

In [ ]:
txt = """
Hagley railway station serves the English village of Hagley, Worcestershire. Off peak trains call three times an hour in each direction, running to or through Kidderminster westwards and through Stourbridge and Birmingham Snow Hill eastwards. Additional trains also call during the morning and evening rush hours. A half hourly service runs in the late evenings and an hourly service on Sundays.[1] Customer Information Screens are installed on either platform. Since the 2017 change of franchise, services are run by West Midlands Trains.[2]

The nearest railway stations are Stourbridge Junction (towards Birmingham) and Blakedown (towards Kidderminster and Worcester).

The station retains one of its GWR-era station buildings and its canopied footbridge, both dating from 1884. Although typical of its era, very few examples of that kind of ornamental ironwork bridge now survive and it was listed grade 2 in 2000.[3] When it was refurbished in late 2011, the colours reverted from its former navy blue and white to the original GWR cream and salmon livery.[4] The footbridge was also used by Hornby as the basis for its 00 Gauge model.[5]

History
The original village of Hagley was a mile away uphill; when its station first appeared in timetables in 1862 as part of the Oxford, Worcester and Wolverhampton railway, it was a rough and ready structure with platforms built of old sleepers. With the line subsequently being taken over by GWR, and the expansion of Lower Hagley along the nearby road to Worcester, there was a demand for a proper building with a station approach up to it.[6] Canopied brick buildings were constructed on either side of the line at this time. While the one on the Stourbridge side housed waiting rooms and toilets, on the Kidderminster and Station Drive side there was the stationmaster's office, the ticket office, and two more waiting rooms and toilets.

Under him the stationmaster had a booking clerk and three porters as well as someone to deliver parcels and personal luggage. There were also three signalmen at the box beyond the Brake Lane bridge and maintenance workers responsible for the track and embankments. The signal box has now gone while the building on the Stourbridge platform has been demolished and replaced with a metal shelter. In the former goods yard north of the station on Brake Lane were coal merchants and the offices responsible for dealing with livestock brought by train for sale at the Monday cattle market, which was located uphill in the old village (at the junction of the A491 and the A456). Where the sidings for coal trucks and the cattle pens used to be, there is now a private housing development named The Sidings after the site.
"""

tokens = tokenizer.encode_plus(txt, add_special_tokens=False)

len(tokens['input_ids'])

In [ ]:
tokens = tokenizer.encode_plus(txt, add_special_tokens=False,
                               retrun_tensors='pt')

print(len(tokens['input_ids'][0]))
tokens

In [ ]:
a = torch.arange(10)
a

In [ ]:
torch.split(a, 4)

In [ ]:
input_id_chunks = tokens['input_ids'][0].split(510)
mask_chunks = tokens['attention_mask'][0].split(510)

In [ ]:
len(input_id_chunks)

In [ ]:
len(input_id_chunks[0])

In [ ]:
len(input_id_chunks[2])

In [ ]:
a = torch.cat(
    [torch.Tensor([101]), a, torch.Tensor([102])]
)

a

In [ ]:
padding_len = 20 - a.shape[0]

padding_len

In [ ]:
if padding_len > 0:
    a = torch.cat(
        [a, torch.Tensor([0] * padding_len)]
    )

a

In [ ]:
type(input_id_chunks)

In [ ]:
chunksize = 512

input_id_chunks = list(tokens['input_ids'][0].split(chunksize - 2))
mask_chunks = list(tokens['attention_mask'][0].split(chunksize - 2))

for i in range(len(input_id_chunks)):
    input_id_chunks[i] = torch.cat(
        [torch.Tensor([101]), input_id_chunks[i], torch.Tensor([102])]
    )
    mask_chunks[i] = torch.cat(
        [torch.Tensor([101]), mask_chunks[i], torch.Tensor([102])]
    )
    
    pad_len = chunksize - input_id_chunks[i].shape[0]
    
    if pad_len > 0:
        input_id_chunks[i] = torch.cat(
            [input_id_chunks[i], torch.Tensor([0] * pad_len)]
        )
        mask_chunks[i] = torch.cat(
            [mask_chunks[i], torch.Tensor([0] * pad_len)]
        )

In [ ]:
for chunk in input_id_chunks:
    print(len(chunk))

In [ ]:
chunk

In [ ]:
input_ids = torch.stack(input_id_chunks)
attention_mask = torch.stack(mask_chunks)

In [ ]:
input_ids

In [ ]:
input_dict = {
    'input_ids': input_ids.long(),
    'attention_mask': attention_mask.int()
}

input_dict

In [ ]:
outputs = model(**input_dict)
probs = torch.nn.functional.softmax(output[0], dim=-1)
probs = probs.mean(dim=0)

In [ ]:
probs

In [ ]:
torch.argmax(probs).item()